In [4]:
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
import pandas as pd
import pickle
import glob
from tqdm.auto import tqdm
import os
from datetime import datetime
import json

In [5]:
# point this at the folder full of csv files from the extraction script
tile_step_folder = '/mnt/2TB Working/Projects/Dam forecast/v4/forecast/time steps'
# the output location for this script
output_folder = '/mnt/2TB Working/Projects/Dam forecast/v4/forecast/preds'
# location of model file
model_folder = '/mnt/2TB Working/Projects/Dam forecast/v4'
model_name = 'dam_forecast_23-11-21.pkl'
model_path = os.path.join(model_folder,model_name)
'found model?',os.path.isfile(model_path)

('found model?', True)

In [6]:
# load in data from meta file
with open(model_path.replace('.pkl','.json'), 'r') as fp:
    meta_dict = json.load(fp)

In [5]:
# load up model
xgb_reg_model = pickle.load(open(model_path, "rb"))

In [6]:
# get a refence to each csv file
time_steps = glob.glob(tile_step_folder+'/*.csv')
len(time_steps)

120

In [7]:
# work out which cols we need to remove, other the model wont know whats happening
cols_to_keep = meta_dict['col_names']
ts_df = pd.read_csv(time_steps[0])
drop_cols = []
for col in ts_df.columns:
    if col not in cols_to_keep:
        drop_cols.append(col)

In [8]:
# loop over each csv file and run model
for time_step in tqdm(time_steps):
#     read csv file
    full_ts_df = pd.read_csv(time_step)
#     drop cols we dont need
    ts_df = full_ts_df.drop(columns=drop_cols)
#     make sure the cols are in the same order as when it was trained
    ts_df = ts_df.reindex(sorted(ts_df.columns), axis=1)
#     run model
    preds = xgb_reg_model.predict(ts_df)
        
#     grab the filename which should also tell us the date
    file_name = os.path.basename(time_step)
    date = file_name.replace('.csv','')    
#     make a new df with just files names as a reference
    limited_df = full_ts_df[['file_name']].copy()
#     place the preds into the new df
    limited_df[date] = preds
#     save the df out to disk
    csv_out_path = os.path.join(output_folder,file_name)
    limited_df.to_csv(csv_out_path)
    

  0%|          | 0/120 [00:00<?, ?it/s]

In [9]:
# get a list of all prediction files
outputs_csvs = glob.glob(output_folder+'/*.csv')
len(outputs_csvs)

120

In [10]:
# open one input csv and read the non weather data
base_data = pd.read_csv(time_steps[0])
drop_list = []
for col in base_data.columns:
    if 'before' in col:
        drop_list.append(col)
    if 'Unnamed' in col:
        drop_list.append(col)
        
base_data = base_data.drop(columns=drop_list)
base_data.head()

,file_name,area,area_2,SRC_DATE,file_name_pred,class,class_name,geometry
0,arcgisonline_1686584.tif,1472.265188,293.852789,20160217.0,arcgisonline_1686584_pred_and_score.tif,0,dam and water,POINT (121.8136426111555 -33.00994748470646)
1,arcgisonline_1686963.tif,3511.531725,1348.727385,20150121.0,arcgisonline_1686963_pred_and_score.tif,0,dam and water,POINT (121.5687803560598 -33.11646584652933)
2,arcgisonline_437392.tif,7400.931025,NaN,20161230.0,arcgisonline_437392_pred_and_score.tif,2,no water,POINT (147.1971736181173 -29.86992984418018)
3,arcgisonline_1747284.tif,2925.907475,2924.952538,20160302.0,arcgisonline_1747284_pred_and_score.tif,1,no dam,POINT (147.4819841980934 -42.99338319262259)
4,arcgisonline_411890.tif,5270.397670,15.310323,20150315.0,arcgisonline_411890_pred_and_score.tif,2,no water,POINT (144.992580889845 -34.25548900296211)


In [11]:
# make empty df to load preds into, we need to keep this separate from the base data we we can sort it by date 
preds_df = pd.DataFrame()
# loop over each pred csv
for csv in tqdm(outputs_csvs):
#     get the file name
    file_name = os.path.basename(csv)
#     get the date from the name
    date = file_name.replace('.csv','')
#     open file
    current_df = pd.read_csv(csv)
#     extract the preds col
    preds = current_df[date].to_list()
#     name date a proper python date
    date_fixed = datetime.strptime(date, '%Y-%m')
#     load preds into main df
    preds_df[date_fixed] = preds
# sort the cols by date
preds_df = preds_df[sorted(preds_df.columns)]

  0%|          | 0/120 [00:00<?, ?it/s]

/home/nick/anaconda3/envs/XGB/lib/python3.7/site-packages/ipykernel_launcher.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  app.launch_new_instance()


In [12]:
# have a look at the df and make sure it looks good
preds_df.head()

,2011-01-01,2011-02-01,2011-03-01,2011-04-01,2011-05-01,2011-06-01,2011-07-01,2011-08-01,2011-09-01,2011-10-01,...,2020-03-01,2020-04-01,2020-05-01,2020-06-01,2020-07-01,2020-08-01,2020-09-01,2020-10-01,2020-11-01,2020-12-01
0,110.85137,158.93771,135.37556,174.14394,248.0782,251.61627,160.77977,170.73775,128.36656,272.68427,...,112.95490,147.74187,158.13924,48.90754,251.98740,254.27902,229.15556,249.95795,130.49240,279.4671
1,986.49870,1004.31180,783.42780,840.63550,637.3890,687.73334,689.98193,913.81793,883.18800,1061.05040,...,660.58887,557.06476,1121.68970,729.68520,679.86444,717.40550,658.72125,723.23830,750.75574,641.2314
2,3771.77700,4333.30960,4195.49850,3063.50370,4462.9375,4086.76730,4482.80500,3314.65200,3450.30790,2962.83500,...,2262.51440,3125.91900,3401.70870,3969.80370,3402.16060,2814.84160,2916.50300,1241.27650,2473.93000,2293.2986
3,1610.13540,1557.64210,1653.89160,1685.15500,1587.2556,1650.95810,1792.34480,1799.76510,1432.66430,1551.19320,...,1036.57710,874.47640,1293.28520,1353.58220,1298.91880,1603.86940,1323.47350,1740.52890,1773.83690,1794.5408
4,3298.70120,3308.74500,3177.32230,3302.80860,2992.1250,2418.55660,2799.33670,2801.64900,2754.41360,2269.65820,...,1143.66580,1918.07070,1957.53260,1832.51100,1462.47900,2087.66240,2190.70700,1690.84410,1627.62730,1221.1123


In [13]:
# join the base data with the preds
preds_with_meta = pd.concat([base_data, preds_df], axis=1)

In [14]:
# export out to csv file
final_export = os.path.join(output_folder,'full preds.csv')
preds_with_meta.to_csv(final_export)